# Working with precipitation files in Python

You will need to:
- Import modules that are important to complete the work.
- Load the files and "understand" them.
- Join the files in a large matrix.
- Compute statistics.
- Save the files for later use.

Optionally:
- Create plots

### Do not forget to run all cells in order, from top to bottom. Failing to do so may cause errors.
### AI Large Language Models (such as deepseek) can help you a lot doing this.

## Modules

Some useful modules are:
- `pandas` (https://pandas.pydata.org/docs/getting_started/intro_tutorials/): to work with tabular data (including import and export). It is the "Microsoft Excel" of Python.
- `matplotlib` (https://matplotlib.org/stable/plot_types/index.html): to create figures (plots).
- `pathlib`: not as important. To handle folder and file paths.

Some syntax examples:
- `import pandas` (this imports the pandas module).
- `import pandas as pd` (this will allow you to write `pd` in your code instead of `pandas` - just more practical).
- `from pandas import read_csv` (this will allow you to import just the `read_csv` function and not the whole of `pandas`.
- `import matplotlib.pyplot as plt` (now we are importing a submodule of `matplotlib` (`pyplot`) as `plt`).
- `from matplotlib import pyplot as plt` (This is another way of doing it). 


## Import modules

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

## Now let's read a CSV file
... and see what it looks like.

Specify where the file is with `Path`.  
`r'.\'` means the path is relative to where this file (the code) is.  
Read the file with `with open(...) as ... :`


In [ ]:
file_path = Path(r'.\Lab work\6606347\precipitation\UKE00105909.csv')

with open(file_path, 'r', encoding='utf-8') as file:
    for i, line in enumerate(file):
        print(line.rstrip())
        if i >= 10:
            break

## Now let's read it with pandas

It is really this easy!

Check out the documentation here:
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [ ]:
pd.read_csv(file_path)

## We can provide more information to the reader too
I wish to use the date as the row index.  
To parse the dates correctly as `datetime64[ns]` we use the `pd.to_datetime` function.   
    You can find all about the format here: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior   
We also want to discard all columns beyond PRCP (precipitation)

### Precipitation data from this source (GHCN-Daily) is stored in 0.1 mm/day. We must convert to mm/day.

In [ ]:
my_data = pd.read_csv(file_path, index_col=1, usecols=[0, 1, 2, 3, 4, 5, 6])
my_data.index = pd.to_datetime(my_data.index, format='%Y-%m-%d', errors='coerce')
my_data.PRCP /= 10

station = my_data.iloc[0, 0]
print(f'This station is {station}')

my_data

## Let's see what the data looks like
Using matplotlib (included in pandas).  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html

In [ ]:
my_data.iloc[:, [-1]].plot()

my_data.loc['2015-01-01':'2015-01-31', ['PRCP']].plot(kind='bar')

ax = my_data.loc['2015-01-01':'2016-01-01', ['PRCP']].plot(linestyle=' ', marker='.', color='red')
_ = ax.set_xlabel('Date')
_ = ax.set_ylabel('Precipitation [mm/day]')


## Let's now aggregate to yearly
We can use the `pd.resample` function:   
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html

`my_data.loc[:, ['PRCP']].resample('YS-APR').sum(min_count=365)`  
This will focus on the "PRCP" column (`,loc()`)  
and resample based on a sum of all values on a yearly basis, with start in april `YS-APR` (just an example).  
Also, we do a sum that only returns values if there are at least 365 entries `.sum(min_count=365)`.  
Finally, all missing data are dropped (`.dropna()`).



In [ ]:
yearly_data = my_data.loc[:, ['PRCP']].resample('YS-APR').sum(min_count=365).dropna()
yearly_data

## Let's compute statistics
We can use .agg

In [ ]:
stats = yearly_data.agg(['mean', 'std', 'count', 'min', 'max', 'skew'])
stats.loc['cv'] = stats.loc['std'] / stats.loc['mean']
stats

## Now let's join, change the header, and export to Excel

In [ ]:
joint = pd.concat([yearly_data, stats], axis=0)
joint.columns = pd.MultiIndex.from_product([['Precipitation [mm/day]'], [station]], names=['Variable', 'Station'])

joint.to_excel(f'parsed_{station}.xlsx')

joint

## Now the magic begins to happen...
Lets do this for all stations at once!

First, use `glob` to get all files with the `.csv` extension in the `precipitation` folder

In [ ]:
folder_path = Path(r'.\Lab work\6606347\precipitation')

for file in folder_path.glob('*.csv'):
    print(file)

In [ ]:
all_data = []
for file in folder_path.glob('*.csv'):
    station = file.name.replace('.csv','')
    _data = pd.read_csv(file, index_col=0, usecols=[1, 6])
    _data.index = pd.to_datetime(_data.index, format='%Y-%m-%d', errors='coerce')
    _data.PRCP /= 10

    _data.loc[:, ['PRCP']].resample('YS-APR').sum(min_count=365).dropna()
    _data.columns = pd.MultiIndex.from_product([['Precipitation [mm/day]'], [station]], names=['Variable', 'Station'])
    
    all_data.append(_data)

full_dataset = pd.concat(all_data, axis=1).dropna()
full_dataset

## Can you add the statistics and save to an Excel file?